<a href="https://colab.research.google.com/github/zangdev/ml_crypto_ml/blob/main/ml_crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
import requests
import time
import os
import joblib


def append_to_file(file_name, content):
    timestamp = pd.Timestamp.now()
    with open(file_name, 'a') as file:
        file.write(f"{timestamp} - {content}\n")


# Function to fetch real-time Bitcoin data from CoinGecko API
def get_bitcoin_data():
    try:
        url = "https://bitcoinaverage-global-bitcoin-index-v1.p.rapidapi.com/indices/global/ticker/BTCUSD"
        headers = {
            "X-RapidAPI-Key": "b204f4c757msh7af6d9244ca8807p166086jsne81ed71fea4a",
            "X-RapidAPI-Host": "bitcoinaverage-global-bitcoin-index-v1.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers)
        data = response.json()
        return process_bitcoin_data(data)
    except Exception as e:
        print("Error fetching Bitcoin data:", e)
        return None


def process_bitcoin_data(data):
    bitcoin_price = data['last']
    bitcoin_high = data['high']
    bitcoin_low = data['low']
    bitcoin_volume = data['volume']
    bitcoin_open_hour = data['open']['hour']
    bitcoin_open_day = data['open']['day']
    bitcoin_open_week = data['open']['week']
    bitcoin_open_month = data['open']['month']
    bitcoin_open_month_3 = data['open']['month_3']
    bitcoin_open_month_6 = data['open']['month_6']
    bitcoin_open_year = data['open']['year']
    bitcoin_changes_hour = data['changes']['price']['hour']
    bitcoin_changes_day = data['changes']['price']['day']
    bitcoin_changes_week = data['changes']['price']['week']
    bitcoin_changes_month = data['changes']['price']['month']
    bitcoin_changes_month_3 = data['changes']['price']['month_3']
    bitcoin_changes_month_6 = data['changes']['price']['month_6']
    bitcoin_changes_year = data['changes']['price']['year']
    bitcoin_volume_percent = data['volume_percent']
    timestamp = data['timestamp']
    return {
        'Price': bitcoin_price,
        'High': bitcoin_high,
        'Low': bitcoin_low,
        'Volume': bitcoin_volume,
        'Open_Hour': bitcoin_open_hour,
        'Open_Day': bitcoin_open_day,
        'Open_Week': bitcoin_open_week,
        'Open_Month': bitcoin_open_month,
        'Open_Month_3': bitcoin_open_month_3,
        'Open_Month_6': bitcoin_open_month_6,
        'Open_Year': bitcoin_open_year,
        'Changes_Hour': bitcoin_changes_hour,
        'Changes_Day': bitcoin_changes_day,
        'Changes_Week': bitcoin_changes_week,
        'Changes_Month': bitcoin_changes_month,
        'Changes_Month_3': bitcoin_changes_month_3,
        'Changes_Month_6': bitcoin_changes_month_6,
        'Changes_Year': bitcoin_changes_year,
        'Volume_Percent': bitcoin_volume_percent,
        'Timestamp': timestamp
    }


# Function to predict future Bitcoin prices using MLPRegressor
def predict_future_prices(bitcoin_data, num_days=1):
    X = bitcoin_data.drop(columns=['Price', 'High', 'Low', 'Volume', 'Volume_Percent'])
    y = bitcoin_data['Price']
    model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500)
    model.fit(X.values, y)
    future_index = len(bitcoin_data) + num_days
    future_price = model.predict([X.iloc[-1]])[0]  # Sử dụng dữ liệu cuối cùng để dự đoán
    return future_price, model


# Function to determine buy, sell, or hold signal
def determine_signal(current_price, predicted_price):
    if predicted_price > current_price:
        return "Buy"
    elif predicted_price < current_price:
        return "Sell"
    else:
        return "Hold"


# Main function for real-time prediction
def real_time_prediction(initial_samples=10, retrain_threshold=20, model_path="bitcoin_model_v3.pkl"):
    try:
        model = joblib.load(model_path)
        print("Model loaded successfully from", model_path)
    except Exception as e:
        print("Error loading model:", e)
        print("Initializing model to None.")
        model = None

    historical_data = pd.DataFrame(columns=['Price', 'High', 'Low', 'Volume', 'Open_Hour', 'Open_Day', 'Open_Week',
                                            'Open_Month', 'Open_Month_3', 'Open_Month_6', 'Open_Year',
                                            'Changes_Hour', 'Changes_Day', 'Changes_Week', 'Changes_Month',
                                            'Changes_Month_3', 'Changes_Month_6', 'Changes_Year', 'Volume_Percent',
                                            'Timestamp'])
    prediction_count = 0
    wrong_predictions = 0
    correct_predictions = 0
    total_predictions = 0

    # Fetch initial historical data for a certain number of samples
    while prediction_count < initial_samples:
        bitcoin_data = get_bitcoin_data()
        if bitcoin_data is not None:
            print("Train sample:", bitcoin_data)
            historical_data = pd.concat([historical_data, pd.DataFrame(bitcoin_data, index=[0])], ignore_index=True)
            prediction_count += 1
            time.sleep(10)  # Wait for 1 second before fetching next sample

    # Enter real-time prediction loop
    while True:
        current_data = get_bitcoin_data()
        if current_data is not None:
            historical_data = pd.concat([historical_data, pd.DataFrame(current_data, index=[0])], ignore_index=True)

            # Predict and determine signal for the current price
            predicted_price, model = predict_future_prices(historical_data)
            current_price = current_data['Price']
            signal = determine_signal(current_price, predicted_price)
            print("Current Bitcoin Price:", current_price)
            print("Predicted Price (10 seconds later):", predicted_price)
            print("Signal:", signal, "| Time:", pd.Timestamp.now())
            total_predictions += 1
            time.sleep(10)

            new_price_check = get_bitcoin_data()['Price']
            print("Price after check:", new_price_check)

            if signal == "Buy":
                if new_price_check > current_price:
                    correct_predictions += 1
                    print("Correct prediction!")
                else:
                    wrong_predictions += 1
                    print("Wrong prediction!")
            elif signal == "Sell":
                if new_price_check < current_price:
                    correct_predictions += 1
                    print("Correct prediction!")
                else:
                    wrong_predictions += 1
                    print("Wrong prediction!")

            # If too many wrong predictions, retrain the model
            if wrong_predictions >= 5:
                print("The prediction: Correct prediction ", correct_predictions, "Wrong prediction ",
                      wrong_predictions)
                print("\nRetraining the model with updated data...\n")
                historical_data = historical_data.iloc[
                                  -retrain_threshold:]  # Keep only the last 'retrain_threshold' samples
                predicted_price, model = predict_future_prices(historical_data)
                signal = determine_signal(current_price, predicted_price)
                print("New Prediction:", "| Predicted Price:", predicted_price, "| Signal:", signal, "| Time:",
                      pd.Timestamp.now())

                file_name = "log.txt"
                content = f"The prediction: Correct {correct_predictions}, Wrong {wrong_predictions}"
                if not os.path.exists(file_name):
                    with open(file_name, 'w') as file:
                        pass
                append_to_file(file_name, content)

                joblib.dump(model, model_path)
                wrong_predictions = 0
                correct_predictions = 0
                total_predictions = 0
                prediction_count = 0

            # If enough correct predictions in the recent past, continue with prediction
            if total_predictions == 10 and correct_predictions >= 5:
                print("\nEnough correct predictions in the recent past. Continuing with prediction...\n")
                wrong_predictions = 0
                correct_predictions = 0
                total_predictions = 0
                prediction_count = 0

        else:
            print("Error fetching current Bitcoin data. Retrying in 10 seconds...")
            time.sleep(10)  # Wait for 10 seconds before retrying


# Start real-time prediction
real_time_prediction()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.4.1.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Model loaded successfully from bitcoin_model_v3.pkl
Train sample: {'Price': 70431.05, 'High': 71536.54, 'Low': 69609.43, 'Volume': 25762.92317842, 'Open_Hour': 70402.57, 'Open_Day': 70792.27, 'Open_Week': 65682.3, 'Open_Month': 61903.0, 'Open_Month_3': 42437.6, 'Open_Month_6': 26958.69, 'Open_Year': 28099.53, 'Changes_Hour': 28.48, 'Changes_Day': -361.21, 'Changes_Week': 4748.76, 'Changes_Month': 8528.05, 'Changes_Month_3': 27993.45, 'Changes_Month_6': 43472.37, 'Changes_Year': 42331.52, 'Volume_Percent': 90.79, 'Timestamp': 1711689593}
Train sample: {'Price': 70430.99, 'High': 71536.54, 'Low': 69609.43, 'Volume': 25762.43432137, 'Open_Hour': 70402.57, 'Open_Day': 70792.27, 'Open_Week': 65682.3, 'Open_Month': 61903.0, 'Open_Month_3': 42437.6, 'Open_Month_6': 26958.69, 'Open_Year': 28099.53, 'Changes_Hour': 28.42, 'Changes_Day': -361.27, 'Changes_Week': 4748.7, 'Changes_Month': 8527.99, 'Changes_Month_3': 27993.39, 'Changes_Month_6': 43472.3, 'Changes_Year': 42331.46, 'Volume_Percent': 